# Market Entrance Game

In [ ]:
import glob
import pandas as pd
import numpy as np
from collections import defaultdict

from plothelpers import Params, extract_params, split_into_runs, entropy
import hmp

from sklearn.metrics import mean_absolute_error 
from scipy.interpolate import make_interp_spline

from statsmodels.tsa.stattools import acf, kpss, adfuller
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar import plotting

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import PolyCollection
plt.style.use('bmh')

In [ ]:
# Where to read the results from
proposed_folder = "../out/general/"
comparison_folder = "../out/comp/"
noise_folder = "../out/noise/"

In [ ]:
def loss(preds, real):
    # Note: sklearn has flipped params (real first)
    return mean_absolute_error(real, preds)

## Sensitivity to c
Visualise methods over entire range of C

In [ ]:
def read_outputs(path, compression=None):
    
    all_results = {}

    for file_name in glob.glob(path):
        runs = pd.read_csv(file_name, index_col=0, compression=compression)
        params = extract_params(file_name) # Based on filename
        
        results = {
            "mean": runs.mean(axis=1),
            "std": runs.std(axis=1),  
            "all": runs,
        }
        
        all_results[params] = results
        
    return all_results

In [ ]:
proposed_results = read_outputs(proposed_folder + "attendance-*")
comparison_results = read_outputs(comparison_folder + "attendance-*")

In [ ]:
def summary_statistics(attendance_rates):
    cs = []
    means = []
    stds = []
    
    for params, results in sorted(attendance_rates.items(), key=lambda x: x[0].c):
        means.append(results["mean"].mean())
        stds.append(results["mean"].std())
        cs.append(params.c)
        
    return np.asarray(means), np.asarray(stds), np.asarray(cs)

In [ ]:
def plot_summary(means, stds, cs, color, label):
    plt.plot(cs, means, c=color, label=label)
    plt.fill_between(cs, means + stds, means - stds, color=color, alpha=0.25)

In [ ]:
proposed_mean, proposed_std, proposed_cs = summary_statistics(proposed_results)
comparison_mean, comparison_std, comparison_cs = summary_statistics(comparison_results)

plot_summary(proposed_mean, proposed_std, proposed_cs, color="purple", label="BRATS")
plot_summary(comparison_mean, comparison_std, comparison_cs, color="orange", label="AS")
    
plt.plot([0,1], [0,1], linestyle="--",c="red", label="Perfect Utilisation")
plt.xlabel("Desirable Entrance Rate")
plt.ylabel("Resulting Entrance Rate")
plt.legend()
plt.savefig("images/attendance.pdf", bbox_inches='tight')
plt.show()

## Resource Efficiency
Visualise the attendance rates for each value of c

In [ ]:
def line(results, params, ax=None, color=None, label=None):
    
    if ax is None:
        ax = plt.gca()
    
    T = len(results["mean"])
    ts = range(T)
    
    ax.plot(results["mean"], label=label, c=color, alpha=0.5)
    
    ax.fill_between(ts, results["mean"] + results["std"], results["mean"] - results["std"],
                     color=color, alpha=0.1)
    
    # Add yticks at c
    yticks = [0, params.c, 1]

    ax.set_yticks(yticks)
    ax.set_yticklabels([str(int(tick * 100)) + "%" for tick in yticks])

    ax.grid(None)
    ax.set_ylim((0,1))

In [ ]:
def loss_statistics(runs, goal):
    losses = [loss(runs[run], goal) for run in runs]
    return {"mean": np.mean(losses), "std": np.std(losses)}

In [ ]:
def compare(proposed, comparison):
    errors = defaultdict(dict)

    for param, proposed_result in sorted(proposed.items(), key=lambda x: x[0].c):
        
        if param not in comparison:
            continue
            
        comparison_result = comparison[param]
        goal = [param.c] * len(proposed_result["mean"])
                
        errors["proposed"][param.c] = loss_statistics(proposed_result["all"], goal)
        errors["comparison"][param.c] = loss_statistics(comparison_result["all"], goal)
            
        # Only plot main points
        if (param.c * 100) % 10 == 0:

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8,4), sharey=True)
            fig.tight_layout()
            
            line(proposed_result, param, ax1, label="BRATS", color="purple")
            line(comparison_result, param, ax2, label="AS", color="orange")
            
            # Add the target line
            ax1.axhline(param.c, ls="--", color="red")
            ax2.axhline(param.c, ls="--", label="Perfect Utilisation", color="red")
            
            # Only do legend on 1 plot
            if param.c == 0.1:
                lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
                lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
                ax2.legend(lines, labels)

            plt.savefig(f"images/attendance/{param.c}.pdf", bbox_inches="tight")
            plt.show()
            
    return errors

In [ ]:
errors = compare(proposed_results, comparison_results)

In [ ]:
def plot_errors(errors, color, label):
    cs = []
    means = []
    stds = []
    for c in sorted(errors):
        # Skip cases where everyone goes or nobody goes
        if c == 0 or c == 1:
            continue

        cs.append(c)

        means.append(errors[c]["mean"])
        stds.append(errors[c]["std"])
        
    means = np.asarray(means)
    stds = np.asarray(stds)

    plt.plot(cs, means,  c=color, label=label)
    plt.fill_between(cs, means - stds, means + stds,  color=color, label=label, alpha=0.1)
    
    return cs

In [ ]:
cs = plot_errors(errors["proposed"], color="purple", label="Proposed")
plot_errors(errors["comparison"], color="orange", label="Comparison")

plt.plot(cs, [0] * len(cs), c="red", ls="--", label="Optimal")
    
plt.ylabel("Mean Absolute Error")
plt.xlabel("Desirable Entracy Rate")
plt.savefig("images/errors.pdf", bbox_inches="tight")

## Clustered Volatility
Compute temporal autocorrelation functions to examine if clustered volatility is present

In [ ]:
n_lags = 5

In [ ]:
def pct_change(x):
    """
        A verison of where percentage change where we deal with case of 0 attendance
    """
    # Take a copy since modifying series
    x = x.copy()
    
    # Since we use pct_change, want to avoid case of 0 attendance, treat as attendance of 1%
    x += 0.01
    
    # Compute change
    pct_change = x.pct_change()
        
    if np.any(~np.isfinite(pct_change[1:])):
        print("Warning, infinite pct_change")
        pct_change[~np.isfinite(pct_change)] = 0
    
    # Exception is first one, which we should just treat as 0 since no change
    pct_change[0] = 0
    
    return pct_change 

def volatility(x):
    return pct_change(x).abs() # Measured as absolute percentage change

In [ ]:
def volatility_correlations(runs, c):
    correlations = []
    confidence_intervals = []

    # Compute for each run
    for run_col in runs:

        run = runs[run_col]
        
        # Compute volatility for the run -- skip first one since always 0
        timeseries = volatility(run)

        # Compute autocorrelation with 95% confidence interval
        corr, confint = acf(timeseries, nlags=n_lags, alpha=0.05, fft=False,)
        
        correlations.append(corr)
        confidence_intervals.append(confint)
        

    # Median over each run
    return np.median(correlations, axis=0), np.median(confidence_intervals, axis=0)

In [ ]:
def read_volatility_correlations(filename, mean_correction=None):
    
    all_correlations = defaultdict(dict)
        
    
    for file in sorted(glob.glob(filename)):
        runs = pd.read_csv(file, index_col=0)
        params = extract_params(file) # Based on filename
        
        if mean_correction:
            runs -= mean_correction
        
        if (params.c == 0 ) or (params.c == 1) or (params.c * 100) % 10 != 0:
            continue

        correlations, confidence_intervals = volatility_correlations(runs, params.c)
        
        all_correlations[params.c]["median"] = correlations
        all_correlations[params.c]["interval"] = confidence_intervals
        
    return all_correlations  

In [ ]:
# Volatility
proposed_volatility_correlations = read_volatility_correlations(proposed_folder + "attendance-*")
comparison_volatility_correlations = read_volatility_correlations(comparison_folder + "attendance-*")
# Noise traders need to be de-meaned since they will always fluctuate around 0.5
noise_volatility_correlations = read_volatility_correlations(noise_folder + "attendance-*", mean_correction=0.5)

In [ ]:
legend = True # Only draw once

for c in proposed_volatility_correlations:
    
    if (c * 100) % 10 != 0:
        continue
        
    correlations = proposed_volatility_correlations[c]
    comparison_correlations = comparison_volatility_correlations[c]
    random_correlations = noise_volatility_correlations[c]
    
    # Plot Correlation
    plt.plot(correlations["median"], c="purple", label="BRATS")
        
    # Plot Confidence intervals
    plt.fill_between(range(n_lags + 1), correlations["interval"][:, 0] - correlations["median"], correlations["interval"][:, 1] - correlations["median"],
                     alpha=0.1, color="gray", zorder=0)
    
    
    if comparison_correlations:
        plt.plot(comparison_correlations["median"], c="orange", label="AS")
        
        # Plot Confidence intervals
        plt.fill_between(range(n_lags + 1), comparison_correlations["interval"][:, 0] - comparison_correlations["median"],
                         comparison_correlations["interval"][:, 1] - comparison_correlations["median"],
                         alpha=0.1, color="gray", zorder=0)

    else:
        print("Warning, no comparison for params")
        
    # Plot random for control/comparison
    if random_correlations:
        plt.plot(random_correlations["median"], linestyle="--", color="gray", label="Noise traders")
        
            # Plot Confidence intervals
        plt.fill_between(range(n_lags + 1), random_correlations["interval"][:, 0] - random_correlations["median"],
                         random_correlations["interval"][:, 1] - random_correlations["median"],
                         alpha=0.1, color="gray", zorder=0)
    else:
        print("Warning, no random comparison for params")

    
    plt.ylabel("Correlation")
    plt.xlabel("Time Lag")

    if legend:
        plt.legend()
        legend = False
        
    plt.savefig(f"images/clustered_volatility/{c}.pdf", bbox_inches='tight')
    plt.show()

## Endogenous crises
Explore the endogenous emergence of crisis, in terms of fat-tails and excess volatility

In [ ]:
def occurence_probability(raw_values):
    combined_dict = {}
    
    for std_range, values in raw_values.items():
        # Combine into one frame
        values = np.hstack(values)

        # Count percentage of trues
        combined_dict[std_range] = 100 * np.sum(values) / len(values)
    
    return combined_dict

In [ ]:
def compute_event_occurences(filename, verbose=True):
    # For counting occurences outside of 1,2,3 stdev
    occurences = {}
    
    for file in sorted(glob.glob(filename)):
        runs = pd.read_csv(file, index_col=0)
        params = extract_params(file) # Based on filename
        
        if params.c == 0 or params.c == 1 or (params.c * 100) % 10 != 0:
            continue

        # Will explore events outside 3 standard deviations
        std_range = [3]

        outside_range = defaultdict(list)        
        
        for run in runs:
            run = runs[run]
                        
            # Look at "returns"
            run = pct_change(run)
            
            mean, std = run.mean(), run.std()

            # Calculate for each of the standard deviation ranges (e.g. 1, 2, 3)
            for std_multiplier in std_range:

                # See how many fall outside the range
                events = np.logical_or(run > (mean + std_multiplier * std), run < (mean - std_multiplier * std))
                                
                # Track events outside
                outside_range[std_multiplier].append(events)
                
        probabilities = occurence_probability(outside_range)   
        
        # Store the average occurences
        occurences[params.c] = {k: round(v, 1) for (k, v) in probabilities.items()}

        if verbose:
            print(params, occurences[params.c])
    
    if verbose:
        print("32, 5, 0.3")
        
    return pd.DataFrame(occurences)

#### Fat-tails
Check the distribution of volatility rates and see if we experience fatter tails than expected

In [ ]:
def combine_dfs(dfs):
    
    # Combine them all. Add index to end so can display in right order (order passed in)
    for idx, df in enumerate(dfs):
        df.columns = [f"{col}_{idx}" for col in df.columns]

    combined_df = pd.concat(dfs, axis=1)
    combined_df = combined_df.reindex(sorted(combined_df.columns), axis=1)
    
    return combined_df

In [ ]:
extreme_event_occurences = compute_event_occurences(proposed_folder + "attendance-*", verbose=False)
extreme_event_occurences_comparison = compute_event_occurences(comparison_folder + "attendance-*", verbose=False)
extreme_event_occurences_noise = compute_event_occurences(noise_folder + "attendance-*", verbose=False)

In [ ]:
df = combine_dfs([extreme_event_occurences, extreme_event_occurences_comparison, extreme_event_occurences_noise])
df.to_csv("images/tail-p.csv")

In [ ]:
df

In [ ]:
def hill_estimator(returns, tail_size):
    # Based on https://www.financialriskforecasting.com/code/MATLABPython9.html
    returns = np.abs(returns) # Assume symettric for computing
    ysort = np.sort(-returns)   # sort the returns
    tail_length = int(len(returns) * tail_size)
    alpha = 1/(np.mean(np.log(ysort[0:tail_length]/ysort[tail_length]))) # get the tail index
    
    return alpha

In [ ]:
def tail_index(runs):
    tail_sizes = [0.025, 0.05, 0.1]
    tail_estimates = defaultdict(list)

    for run in runs:
        run = runs[run]

        # Look at "returns" - i.e., change in attendance
        run = pct_change(run)

        for tail_size in tail_sizes:
            estimate = hill_estimator(run, tail_size=tail_size)
            tail_estimates[tail_size].append(estimate)

    # Store the average occurences
    return {tail_size: round(np.median(estimates), 1) for tail_size, estimates in tail_estimates.items()}


In [ ]:
def compute_tail_shapes(filename):
    # For tail estimates
    occurences = {}
    
    for file in sorted(glob.glob(filename)):
        runs = pd.read_csv(file, index_col=0)
        params = extract_params(file) # Based on filename
        
        if params.c == 0 or params.c == 1 or params.c == 1 or (params.c * 100) % 10 != 0:
            continue     
             
        # Store the average occurences
        occurences[params.c] = tail_index(runs)
                              
    return pd.DataFrame(occurences)

In [ ]:
proposed_tail_index = compute_tail_shapes(proposed_folder + "attendance-*")
comparison_tail_index = compute_tail_shapes(comparison_folder + "attendance-*")
noise_tail_index = compute_tail_shapes(noise_folder + "attendance-*")

In [ ]:
df = combine_dfs([proposed_tail_index, comparison_tail_index, noise_tail_index])
df.to_csv("images/tail.csv")

In [ ]:
df = pd.DataFrame({
    "BRATS": proposed_tail_index.values.flatten(),
    "AS": comparison_tail_index.values.flatten(),
    "Noise Traders": noise_tail_index.values.flatten()
})

In [ ]:
colours = ["purple", "orange", "gray"]
ax = sns.violinplot(data=df, color="gray",)

for idx, violin in enumerate(ax.findobj(PolyCollection)):
    color = colours[idx]
    violin.set_edgecolor(color)
    violin.set_facecolor(color)
    violin.set_alpha(0.5)
    
for idx, line in enumerate(ax.get_lines()):
    if idx % 2 != 0:
        idx = idx -1
    color = colours[idx // 2]
    line.set_color(color)

plt.axhline(1, ls="--", c="gray")
plt.axhline(4, ls="--", c="gray")
plt.xlabel("Method")
plt.ylabel(r"Tail Index ($\alpha$)")

plt.savefig("images/violin.pdf", bbox_inches="tight")

# Underlying causes
In this section we look to perform econometric analysis to analyse underlying causes for the stylized facts observed above

#### Setup data

In [ ]:
param_timeseries = {}

attendance = read_outputs(proposed_folder + "attendance-*")
for file in glob.glob(proposed_folder + "resources-*"):
    params = extract_params(file) # Based on filename
    
    if params.c == 0 or params.c == 1 or (params.c * 100) % 10 != 0:
        continue
    
    results = pd.read_csv(file, index_col=0)
    runs = split_into_runs(results, params)

    entropies = []
    volatilities = []

    for run_idx, run in enumerate(runs):
        attendance_run = attendance[params]["all"][str(run_idx)]

        # Differencing in volatility -- as volatility not necessarily stationary
        observed_volatility = volatility(attendance_run).diff()[1:]
        volatilities.append(observed_volatility)

        # Differencing in population entropt 
        ent = run.apply(entropy)
        ent = ent.diff()[2:] # 2, as first one is 0, and second must be skipped because of diff above

        entropies.append(ent)


    param_timeseries[params] = {
        "attendance": attendance[params]["all"],
        "volatility":volatilities,
        "entropy": entropies,
    }

In [ ]:
results = {}
results[proposed_folder] = param_timeseries

In [ ]:
def bitstring_to_numeric(bitstring):
    # Convert a bit string into its numeric representation
    return bitstring.dot(2**np.arange(bitstring.size)[::-1])

def comparison_timeseries(path, comparison_attendance):
    comparison_param_timeseries = {}

    for file in glob.glob(path):
        params = extract_params(file) # Based on filename
        
        if params.c == 0 or params.c == 1 or (params.c * 100) % 10 != 0:
            continue

        results = pd.read_csv(file, index_col=0, compression="gzip")
        all_strategies = split_into_runs(results, params)
        
        attendance_runs = []
        volatilities = []
        entropies = []

        for run_idx, run in enumerate(all_strategies):

            attendance_run = comparison_attendance[params]["all"][str(run_idx)]
            attendance_runs.append(attendance_run)
            
            vectors = []
            strategy_entropies = []
            
            for t in run:
                time = int(t)

                # Best strategy for the previous timestep
                strategy_space = run[str(time - 1) if time > 0 else "0"]
                
                # Read each as list, then convert to numpy
                strategy_space = np.vstack(strategy_space.apply(lambda x: eval(x)))

                # -1 as there is also the bias term
                M = strategy_space.shape[1] - 1

                # Only do it with enough history
                if time >= M:
                    history = attendance_run[time - M: time].values
                    history = history[::-1] # netlogo treats most recent first, so reverse order
                    history = history * params.N # Also convert it back to number
                    
                    # First col is the bias/constant
                    bias = strategy_space[:,0]
                    constant = (params.N * bias)
                    
                    # Rest of strategy as multipliers on history
                    weights = strategy_space[:,1:]
                    weighted_history = weights * history
                    
                    # Combine them
                    strategy_vector = np.column_stack((weighted_history, constant))
                                        
                    # Convert to bitstring and compute entropy
                    agent_bitstrings = strategy_vector >= 0
                    numeric_representations = np.asarray([bitstring_to_numeric(bitstring) for bitstring in agent_bitstrings])
                    ent = entropy(numeric_representations)
                    strategy_entropies.append(ent)
                    
            strategy_entropy = pd.Series(strategy_entropies).diff()[1:]
            entropies.append(strategy_entropy)

            observed_volatility = volatility(attendance_run).diff()[M:]
            volatilities.append(observed_volatility)

        comparison_param_timeseries[params] = {
            "attendance": attendance_runs,
            "volatility":volatilities,
            "entropy": entropies,
        }
        
    return comparison_param_timeseries

In [ ]:
comparison_attendance = read_outputs(comparison_folder + "attendance-*")
comparison_param_timeseries = comparison_timeseries(comparison_folder + "strategies-*", comparison_attendance)

In [ ]:
results[comparison_folder] = comparison_param_timeseries

### Check stationarity
Before doing anything, we test for stationarity of the data. We check for unit root with Augmented Dickey-Fuller test, then stationary with Kwiatkowski–Phillips–Schmidt–Shin tests

In [ ]:
def stationarity_check(timeseries_runs):
    # Check unit root
    unit_root_p_val = [adfuller(timeseries)[1] for timeseries in timeseries_runs]
    # Check stationarity
    non_stationarity_p_val = [kpss(timeseries)[1] for timeseries in timeseries_runs]
    
    # Combine p-values from each of the runs
    return {
        "Unit root":  hmp.hmp(unit_root_p_val),
        "Non-Stationarity":  hmp.hmp(non_stationarity_p_val),
    }

In [ ]:
import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter('ignore', InterpolationWarning)

In [ ]:
for folder, timeseries_dict in results.items():
    
    print(folder)
    
    for params, result in timeseries_dict.items():
        print("Entropy:", stationarity_check(param_timeseries[params]["entropy"]))
        print("Volatility:", stationarity_check(param_timeseries[params]["volatility"]))

### Determine lag-order

M appears to be a good option for the lag-order a priori, as this specifies the history length of the agents. Here, we verify that using M as leg lengths is a good option by computing the Akaike information criterion (AIC) for all lags up to 100 -- and show that AIC is minimised at or around M

In [ ]:
def compute_ranks(param_results):
    
    param_ranks = {}
    
    for params, result in param_results.items():
    
        population_entropies = result["entropy"]
        all_lags = []

        for run_idx, population_entropy in enumerate(population_entropies):

            observed_volatility = result["volatility"][run_idx]

            df = pd.DataFrame({"vol": list(observed_volatility), "ent": population_entropy})
            df.index = pd.RangeIndex(start=0, stop=len(population_entropy))

            model = VAR(df)

            # Select appropriate lags
            selections = model.select_order(maxlags=100)
            aics = selections.ics["aic"] # Information criteria for each lag
            lag_ranks = pd.Series(aics).rank() # rank 1 = lowest AIC
            all_lags.append(lag_ranks)

        param_ranks[params] = np.asarray(all_lags)
        
    return param_ranks

In [ ]:
param_rankings = compute_ranks(results[proposed_folder])
comparison_param_rankings = compute_ranks(results[comparison_folder])

In [ ]:
def plot_ranks(lag_ranks, color):
    lag_ranks = np.asarray(lag_ranks)
    q1_rank, median_rank, q3_rank = np.percentile(lag_ranks, q=[25,50, 75], axis=0)

    plt.plot(median_rank, color=color)
    plt.fill_between(range(len(median_rank)), q1_rank, q3_rank, alpha=0.2, color=color)

In [ ]:
for params, lag_ranks in param_rankings.items():
    
    comparison_lag_ranks = comparison_param_rankings[params]
    
    plot_ranks(lag_ranks, color="purple")
    plot_ranks(comparison_lag_ranks, color="orange")
    
    plt.xlabel("Lags")
    plt.ylabel("AIC Rank")
    plt.savefig(f"images/lags/{params.c}.pdf", bbox_inches='tight')
    plt.show()

In [ ]:
optimal_lag_order = {
    proposed_folder: {},
    comparison_folder: {},
}

for params, lag_ranks in param_rankings.items():
    
    optimal_lag_order[proposed_folder][params.c] = np.argmin(lag_ranks, axis=1).astype(int)
    optimal_lag_order[comparison_folder][params.c] = np.argmin(comparison_lag_ranks, axis=1).astype(int)

### Granger Causality
Now we have verified the lag order, lets check for granger causality

In [ ]:
for folder, timeseries_dict in results.items():
    print(folder)
    
    for params, result in timeseries_dict.items():
        
        if params.c == 0 or params.c == 1:
            continue

        population_entropies = result["entropy"]
        p_values = []

        for run_idx, population_entropy in enumerate(population_entropies):

            observed_volatility = result["volatility"][run_idx]
            df = pd.DataFrame({"vol": list(observed_volatility), "ent": population_entropy})
            df.index = pd.RangeIndex(start=0, stop=len(population_entropy))

            model = VAR(df)
            
            lag_order = optimal_lag_order[folder][params.c][run_idx]
            model_results = model.fit(lag_order)

            # See if entropy granger-causes volatility
            granger = model_results.test_causality("vol", "ent", kind='f')
            p_values.append(granger.pvalue)

        #plt.hist(p_values, bins=10)
        #plt.show()

        # Combine p values
        significance = hmp.hmp(p_values)
        print(params, significance)

### Impulse response analysis

In [ ]:
def irf_values(values, impulse, response, names):
    """
    Reusable function to make flexible grid plots of impulse responses and
    comulative effects
    """
    # We only plot one, extract the indices for the impulse/response from names
    _, _, to_plot = plotting._get_irf_plot_config(names, impulse, response)
    
    # Just the index of our impulse and response
    (j, i, _, _) = to_plot[0]
    
    return values[:, i, j]

In [ ]:
def BSpline(effect):
    """
        Use cubic BSpline to smooth the impulse responses
    """
    T = range(len(effect))
    xnew = np.linspace(0, len(T)-1, 1000) 

    spl = make_interp_spline(T, effect, k=3) # Cubic BSpline
    smoothed = spl(xnew)
    
    return xnew, smoothed

In [ ]:
def plot_smooth_response(effects, color):
    q1_effect, median_effect, q3_effect = np.percentile(effects, q=[25,50, 75], axis=0)
    
    # All x's will be same
    x, smoothed_q1 = BSpline(q1_effect)
    x, smoothed_median = BSpline(median_effect)
    x, smoothed_q3 = BSpline(q3_effect)

    plt.plot(x, smoothed_median, c=color)
    plt.fill_between(x, smoothed_q1, smoothed_q3, alpha=0.2, color=color)

In [ ]:
def compute_impulse_responses(results, folder):

    param_results = results[folder]
    param_effects = {}
    
    for params, result in param_results.items():
        
        if params.c == 0 or params.c == 1:
            continue

        population_entropies = result["entropy"]
        effects = []

        for run_idx, population_entropy in enumerate(population_entropies):

            observed_volatility = result["volatility"][run_idx]

            df = pd.DataFrame({"vol": list(observed_volatility), "ent": population_entropy})
            df.index = pd.RangeIndex(start=0, stop=len(population_entropy))

            model = VAR(df)
            lag_order = optimal_lag_order[folder][params.c][run_idx]
            model_results = model.fit(lag_order)
            irf = model_results.irf(periods=20)

            effect = irf_values(irf.irfs, impulse='ent', response='vol', names=irf.model.names)
            effects.append(effect)
            
        param_effects[params] = np.asarray(effects)
            
    return param_effects

In [ ]:
proposed_responses = compute_impulse_responses(results, proposed_folder)
comparison_responses = compute_impulse_responses(results, comparison_folder)

In [ ]:
for params, effects in proposed_responses.items():
    
    comparison_effects = comparison_responses[params]
    
    plot_smooth_response(effects, color="purple")
    plot_smooth_response(comparison_effects, color="orange")

    plt.xticks(range(0, effects.shape[1], 5))
    plt.savefig(f"images/irf/{params.c}.pdf", bbox_inches='tight')
    plt.show()

    print("------")